## Základné nastavenia

Pri analýze dát budeme používať viacero knižníc, ktoré je na začiatku potrebné naimportovať do Vášho zápisníka. Knižnica `numpy` poskytuje základné typy, ako sú napr. dátové polia reprezentujúce numerické vektory hodnôt, alebo matice. Pri analýze dát budeme najčastejšie pracovať s knižnicou `pandas`, ktorá rozširuje `numpy` a definuje základné typy pre reprezentáciu dátových tabuliek a ich stĺpcov. Knižnice `mathplotlib` a `seaborn` budeme používať na vykresľovanie grafov, ktoré sa zobrazia priamo ako súčasť zápisníka.

In [ ]:
# importujeme potrebné knižnice, tento odstavec by mal byť spustený ako prvý predtým ako sa budú používať
# importované objekty pd, np, plt a sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# nastavíme zobrazovanie grafov priamo v odstavcoch zápisníka
%matplotlib inline
# inicializujeme knižnicu seaborn
sns.set()

## Dátová množina - Správa o šťastí sveta

Správu o šťastí sveta vydáva každoročne Organizácia Spojených Národov a je založená na dátach dotazníkového prieskumu, ktoré sú zozbierané z reprezentatívnej vzorky obyvateľov jednotlivých krajín. Dotazníkový prieskum sa zameriava na hodnotenie vlastného života na základe viacerých kritérií, ktoré zahŕňajú ekonomické ukazovatele, sociálnu starostlivosť, zdravotníctvo, slobodu rozhodovania, vnímanie dobrosrdečnosti medzi ľuďmi a vnímanie korupcie.

![World Happiness Record](whr-2017-happy.jpg)
[Zdroj](http://unsdsn.org/news/2017/03/20/fifth-world-happiness-report-2017-ranks-happiest-countries/)

Dáta ktoré máte k dispozícii sú vyhodnotené za roky 2015, 2016 a 2017 a sú popísané nasledujúcimi atribútmi:

* `Country` - Názov krajiny
* `Region` - Geopolitický región krajiny
* `Rank` - Celkové umiestnenie v rebríčku krajín za daný rok, podľa dosiahnutého skóre
* `Score` - Dosiahnuté skóre vyjadrujúce celkové šťastie v krajine
* `GDP` - Ekonomická situácia - hrubý národný produkt na obyvateľa
* `Family` - Sociálna podpora rodiny
* `Health` - Stav zdravotníctva - očakávaná dĺžka života
* `Freedom` - Sloboda prejavu a rozhodovania
* `Trust` - Vnímanie korupcie
* `Generosity` - Vnímanie dobrosrdečnosti

Do celkového hodnotenia je zahrnutý rozdiel medzi danou krajinou a _Distópiou_ - hypotetickou krajinou, ktorá má najnižšie hodnotenie pre všetky ukazovatele (tzn. žiadna krajina na tom nemôže byť horšie ako Distópia).

### Načítanie dát a základné štatistiky

Dátová tabuľka je v knižnici `pandas` reprezentovaná typom `DataFrame` (_dátový rámec_), ktorý je zložený z jednotlivých stĺpcov reprezentovaných typom `Series` (_dátová postupnosť_). `pandas` priamo podporuje načítanie a zápis dát v štandardných formátoch ako napr. `.csv` (_Comma-Separated Values_ - textový formát s hodnotami oddelenými čiarkou).

In [ ]:
# načítame dáta za rok 2015 zo súboru 2015.csv
data_2015 = pd.read_csv("../data/2015.csv")

# zobrazíme si názvy stĺpcov (vlasnosť columns je objekt typu pandas.Index, ktorý reprezentuje postupnosť indexov pre
# riadky, alebo stĺpce tabuľky, pristupuje sa k nemu podobne ako ku zoznamu)
data_2015.columns

K dátam sa pristupuje rovnako ako pri zoznamoch, alebo mapách indexovaním. Stĺpce sú štandardne indexované podľa ich názvu a hodnoty v stĺpcoch podľa číselného indexu riadku tabuľky (od 0). Pre každý číselný stĺpec môžeme priamo vypočítať základné štatistiky, ako napr. počet neprázdnych hodnôt, minimum, maximum, priemer, štandardnú odchýlku a kvantily.

In [ ]:
# rozmer dátovej tabuľky zistíme z vlastnosti shape
print(data_2015.shape[0]) # prvý rozmer - počet riadkov
print(data_2015.shape[1]) # druhý rozmer - počet stĺpcov

# k dátam pristupujeme indexovaním, napr. vypíšeme hodnotu stĺpca 'Country' na prvom riadku tabuľky (index 0)
print(data_2015['Country'][0])

scores = data_2015['Score'] # scores je objekt typu pandas.Series, ktorý reprezentuje jeden stĺpec tabuľky

# pre číselné stĺpce môžeme vypočítať priamo základné štatistiky
score_count = scores.count()      # počet neprázdnych hodnôt
score_mean = scores.mean()        # priemerná hodnota
score_std = scores.std()          # štandardná odchýlka výberu
score_min = scores.min()          # minimálna hodnota
score_max = scores.max()          # maximálna hodnota
score_q25 = scores.quantile(0.25) # 25% kvartil
score_q50 = scores.quantile(0.5)  # 50% kvartil - medián
score_q75 = scores.quantile(0.75) # 75% kvartil

# štatistiky pre všetky číselné atribúty tabuľky môžeme vypočítať aj naraz pomocou metódy describe()
data_2015.describe()

In [ ]:
# pre kategorické atribúty môžeme zobraziť zoznam rôznych hodnôt a ich početnosti
# napr. pre 'Region'
data_2015['Region'].value_counts()

### Príprava dát

Postupne načítame a upravíme dáta pre roky 2015, 2016 a 2017. Z tabuľky za rok 2015 odstránime stĺpec `StdError`, ktorý udáva štandardnú odchýlku odpovedí medzi obyvateľmi jedného štátu a ďalej ho nebudeme pri analýze používať.

In [ ]:
# stĺpce sa odstráňujú pomocou metódy drop
# štandardne metóda drop nezmení pôvodný dátový rámec, ale vytvorí nový, takže si zmeníme premennu data_2015
# na novú hodnotu
data_2015 = data_2015.drop(columns="StdError")
# do tabuľky pridáme stĺpec 'Year', ktorého hodnotu nastavíme na 2015 pre všetky riadky
data_2015["Year"] = 2015
# pomocou metódy head si zobrazíme prvé riadky tabuľky (štandardne metóda head vracia nový objekt DataFrame s prvými
# 5 riadkami pôvodnej tabuľky)
data_2015.head()

Načítame dáta za rok 2016 z ktorých odstránime stĺpce `LowerConfidence` a `UpperConfidence` a pridáme stĺpec `Year` s hodnotou 2016 pre všetky riadky.

In [ ]:
# načítame dáta za rok 2016
data_2016 = pd.read_csv("../data/2016.csv")
# ak v metóde drop nastavíme parameter inplace na True, stĺpec sa odstráni priamo v pôvodnom rámci a nevytvorí sa
# nový, ako parameter columns môžete naraz zadať zoznam viacerých stĺpcov, ktoré sa majú odstrániť
data_2016.drop(columns=["LowerConfidence", "UpperConfidence"], inplace=True)
# pridáme stĺpec Year
data_2016["Year"] = 2016

# pre výpis riadkov môžete pre metódu head zadať počet riadkov, ktoré sa majú zobraziť
data_2016.head(1) # vypíšeme 1 riadok

Z výpisu si môžete všimnúť, že tabuľka za rok 2016 neobsahuje celkové skóre a poradie krajín (stĺpce `Score` a `Rank`). Keďže `Score` a `Rank` sú vypočítané spočítaním ostatných ukazovateľov, môžeme ich jednoducho dopočítať metódou `eval`. Pre metódu `eval` sa výraz pre výpočet hodnôt zadáva ako reťazec, ktorý môže obsahovať názvy existujúcich stĺpcov, konštanty (čísla, reťazce a pod.) a operátory `+`, `-`, `*`, `/`, `**`, `%`.

In [ ]:
# pridáme do tabuľky nový stĺpec Score, ktorého hodnotu pre každý riadok vypočítame ako súčet atribútov GDP, Family,
# Health, Freedom, Trust, Generosity a Dystopia
data_2016["Score"] = data_2016.eval("GDP + Family + Health + Freedom + Trust + Generosity + Dystopia")

# stĺpec Rank sa určuje podľa celkového poradia krajín zoradených podľa skóre od najväčšieho po najmenšie
# zoradíme si riadky tabuľky podľa skóre v zostupnom poradí
data_2016.sort_values(by="Score", ascending=False)
# pre kontrolu si zobrazíme 3 prvé a 3 posledné krajiny
data_2016.head(3)

In [ ]:
# metóda tail vracia posledné riadky tabuľky
data_2016.tail(3)

In [ ]:
# keďže dáta máme zoradené zostupne podľa celkového skóre, hodnoty v stĺpci poradia Rank nastavíme na 
# postupnosť čísel 1, 2, ..., počet krajín (počet riadkov v tabuľke)
data_2016["Rank"] = range(1, data_2016.shape[0] + 1)
data_2016.head()

### Úloha 6.1

1. Do premennej `data_2017` načítajte dáta za rok 2017 zo súboru `2017.csv`.
2. Odstráňte nepotrebné stĺpce `WhiskerHigh` a `WhiskerLow`.
3. Pridajte stĺpec `Year` a nastavte jeho hodnotu na 2017 pre všetky riadky.
4. Zobrazte prvý riadok tabuľky.

In [ ]:
data_2017 = None

In [ ]:
# pomocou metódy concat si spojíme dáta za všetky roky do jednej tabuľky, parameter sort=False a ignore_index=True
# udáva, že sa riadky nemajú preusporiadať podľa ich pôvodných indexov a že sa zachová poradanie v akom sú uvedené
# spájané tabuľky (tzn. najprv všetky riadky 2015, potom 2016 a na koniec 2017)
data = pd.concat([data_2015, data_2016, data_2017], ignore_index=True, sort=False)
data.head(1)

### Výber dát

Z tabuľky je možné vybrať iba niektoré stĺpce podľa názvu, alebo riadky podľa zadanej podmienky. Pre výber riadkov môžete použiť metódu `query` pri ktorej sa logická podmienka zadáva jednoducho ako reťazec podobne ako pri zadávaní výrazov v metóde `eval`.

In [ ]:
# vyberieme iba stĺpce Contry, Rank, Score a Year
selected = data[["Country", "Rank", "Score", "Year"]]

# vyberieme iba riadky o Slovensku
selected.query("Country == 'Slovakia'")

In [ ]:
# pri filtrovaní riadkov môžete zadať zložitejšiu podmienku s operátormi <, >, <=, >=, !=, ==, in [zoznam],
# not in [zoznam] a s logickými spojkami and, or a not

# napr. odfiltrujeme riadky o našich susedoch za rok 2017 a výsledok usporiadame podľa celkového poradia krajín
q = "Country in ['Slovakia', 'Czech Republic', 'Poland', 'Hungary', 'Ukraine', 'Austria'] and Year == 2017"
selected.query(q).sort_values(by="Rank")

## Závislosti medzi atribútmi

Jednou zo základných úloh vo fáze pochopenia dát je analyzovať závislosti medzi dvojicami atribútov.

### Závislosti medzi kategorickými a číselnými atribútmi

Závislosti medzi kategorickými a číselnými atribútmi môžeme skúmať napr. pomocou _kontingenčnej tabuľky_, ktorá rozdelí dáta do skupín podľa hodnôt kategorických atribútov a zosumarizuje pre každú skupinu hodnoty číselných atribútov pomocou rôznych agregačných funkcií (napr. priemerná/minimálna/maximálne hodnota, štandardná odchýlka výberu atď.)

Kontingenčnú tabuľku vytvoríme pomocou metódy `pivot_table`, kde musíme okrem dát zadať aspoň jeden kategorický atribút a jeden číselný.

In [ ]:
# napr. na výpočet priemerného skóre pre všetky krajiny v danom roku môžeme zadať
pd.pivot_table(data, index="Year", values="Score")

In [ ]:
# dáta môžeme zoskupiť podľa viacerých kategorických atribútov naraz,
# napr. podľa regiónu a roku
pd.pivot_table(data, index=["Region", "Year"], values="Score")

In [ ]:
# kontingenčnú tabuľku môžeme preusporiadať presunutím niektorých kategorických atribútov z riadkov tabuľky
# (parameter index) na stĺpce (parameter columns)
# napr. predchádzajúcu tabuľku môžeme prehľadnejšie zobraziť nasledovne
table = pd.pivot_table(data, index="Region", columns="Year", values="Score")
table

In [ ]:
# hodnoty kontingenčnej tabuľky si môžeme priamo graficky zobraziť napr. ako horizontálny stĺpcový graf
table.plot(kind="barh")
# nastavíme popis x-ovej osy
l = plt.xlabel("Average Happiness Rank")

In [ ]:
# v jednej tabuľke môžete vypočítať viacero agregačných funkcií nastavením parametra aggfunc na zoznam funkcií,
# napr. pre výpočet priemernej hodnoty a štandardnej odchýlky skóre pre každý región:
pd.pivot_table(data, index="Region", values="Score", aggfunc=["mean", "std"])

In [ ]:
# môžete naraz vypočítať aj rôzne agregačné funkcie pre rôzne číselné atribúty
# napr. v nasledujúcej tabuľke vypočítame priemernú hodnotu pre skóre (Score) a minimálnu a maximálnu hodnotu
# pre poradie (Rank) pre každý región
pd.pivot_table(data, index="Region", values=["Score", "Rank"], aggfunc={"Score": "mean", "Rank": ["min", "max"]})

### Závislosti medzi číselnými atribútmi - korelácia

Základným typom závislosti medzi dvoma číselnými atribútmi je lineárna závislosť, ktorú je možné vyjadriť (_Pearsonovym_) korelačným koeficientom.

In [ ]:
# vyberieme si iba číselné atribúty ukazovateľov
factors = data[["GDP", "Family", "Health", "Freedom", "Trust",  "Generosity"]]
# pomocou metódy corr vypočítame korelačnú tabuľku
corr_table = factors.corr()
corr_table

Hodnoty si môžeme graficky zobraziť pomocou grafu typu _teplotnej mapy_ (_heat map_).

In [ ]:
p = sns.heatmap(corr_table,
                xticklabels=corr_table.columns, yticklabels=corr_table.columns, # označíme osy názvami stĺpcov
                vmin=-1, vmax=1, # nastavíme minimálnu a maximálnu hodnotu pre farebnú paletu
                cmap='coolwarm', # zmeníme prednastavenú paletu farieb 
                square=True)     # zobrazia sa štvorcové polia

Najväčšia závislosť je medzi atribútmi `GDP` a `Health`. Dáta si zobrazíme na X-Y grafe do ktorého zobrazíme aj regresnú priamku.

In [ ]:
p = sns.lmplot(data=factors, x='GDP', y='Health', fit_reg=True) 

Závislosť medzi všetkými číselnými atribútmi naraz si môžete prehľadne zobraziť grafom typu _pair plot_.

In [ ]:
p = sns.pairplot(factors)

## Dátová množina - Iris

Dátová množina **Iris** popisuje kvety kosatcov druhu Iris setosa, Iris virginica a Iris versicolor.
Množinu zozbieral biológ Edgar Anderson a prvý krát ju použil pri dátovej analýze britský štatistik
[Ronald Fisher](https://en.wikipedia.org/wiki/Ronald_Fisher) v roku 1936. Množina obsahuje 50 príkladov z každého druhu, ktoré sú popísané 4 atribútmi:
dĺžkou a šírkou okvetných listov (`petal_width`, `petal_height`) a dĺžkov a šírkov listov kalicha (`sepal_width`, `sepal_height`).

![Iris versicolor](iris.jpg)
[Zdroj](https://en.wikipedia.org/wiki/Iris_versicolor)

### Úloha 6.2

1. Načítajte dáta z dátovej množiny Iris (súbor `../data/iris.csv`) do premennej `iris`.
2. Vypočítajte základné štatistiky pre číselné atribúty.
3. Pre nominálny atribút `species` zobrazte rôzne hodnoty a ich početnosti.
4. Zistite, či dátová množina obsahuje chýbajúce hodnoty.

### Úloha 6.3

Zobrazte graf _pair plot_ pre číselné atribúty. Viete z priebehu určiť, ktoré atribúty sú najviac korelované?

### Úloha 6.4

Vypočítajte korelačnú maticu a zobrazte ju pomocou teplotnej mapy. Určite, ktoré dva atribúty sú najviac a najmenej korelované a zobrazte ich X-Y priebeh spolu s regresnou priamkou.

### Úloha 6.5

Pomocou kontingenčnej tabuľky vypočítajte priemernú hodnotu a štandardnú odchýlku pre atribúty `sepal_length` a `sepal_width` zoskupené podľa atribútu `species`.

### Úloha 6.6

1. Odfiltrujte z dátovej množiny iba riadky (príklady) patriace do triedy `virginica`.
2. Z odfiltrovaných riadkov vytvorte tabuľku s atribútmi `sepal_ratio = sepal_length / sepal_width` a `petal_ratio = petal_length / petal_width`.
3. Zobrazte X-Y závislosť medzi `sepal_ratio` a `petal_ratio` a vypočítajte korelačný koeficient medzi nimi.